In [282]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import statsmodels.api as sm
import seaborn as sns
import re
from scipy.stats import shapiro
sns.set()
from pylab import rcParams
plt.style.use('fivethirtyeight')
sns.set_context("notebook")
rcParams['figure.figsize'] = 15,10

In [283]:
df = pd.read_csv("nyc-rolling-sales.csv")
df.head()

,Unnamed: 0,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,4,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392,6,,C2,153 AVENUE B,...,5,0,5,1633,6440,1900,2,C2,6625000,2017-07-19 00:00:00
1,5,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,26,,C7,234 EAST 4TH STREET,...,28,3,31,4616,18690,1900,2,C7,-,2016-12-14 00:00:00
2,6,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,39,,C7,197 EAST 3RD STREET,...,16,1,17,2212,7803,1900,2,C7,-,2016-12-09 00:00:00
3,7,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,402,21,,C4,154 EAST 7TH STREET,...,10,0,10,2272,6794,1913,2,C4,3936272,2016-09-23 00:00:00
4,8,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,404,55,,C2,301 EAST 10TH STREET,...,6,0,6,2369,4615,1900,2,C2,8000000,2016-11-17 00:00:00


In [284]:
df.shape

(84548, 22)

In [285]:
df.describe()

,Unnamed: 0,BOROUGH,BLOCK,LOT,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,YEAR BUILT,TAX CLASS AT TIME OF SALE
count,84548.000000,84548.000000,84548.000000,84548.000000,84548.000000,84548.000000,84548.000000,84548.000000,84548.000000,84548.000000
mean,10344.359878,2.998758,4237.218976,376.224015,10731.991614,2.025264,0.193559,2.249184,1789.322976,1.657485
std,7151.779436,1.289790,3568.263407,658.136814,1290.879147,16.721037,8.713183,18.972584,537.344993,0.819341
min,4.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,4231.000000,2.000000,1322.750000,22.000000,10305.000000,0.000000,0.000000,1.000000,1920.000000,1.000000
50%,8942.000000,3.000000,3311.000000,50.000000,11209.000000,1.000000,0.000000,1.000000,1940.000000,2.000000
75%,15987.250000,4.000000,6281.000000,1001.000000,11357.000000,2.000000,0.000000,2.000000,1965.000000,2.000000
max,26739.000000,5.000000,16322.000000,9106.000000,11694.000000,1844.000000,2261.000000,2261.000000,2017.000000,4.000000


#### Dropping Columns:
- 1st column that is just indexes is useless
- EAST-MENT is totally empty
- APARTMENT NUMBER has 65,496 missing values and it is useless

In [286]:
df["APARTMENT NUMBER"][df["APARTMENT NUMBER"] != " "].count()

19052

In [287]:
df = df.drop(["Unnamed: 0", "EASE-MENT", "APARTMENT NUMBER"], axis='columns')
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,BUILDING CLASS AT PRESENT,ADDRESS,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392,6,C2,153 AVENUE B,10009,5,0,5,1633,6440,1900,2,C2,6625000,2017-07-19 00:00:00
1,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,26,C7,234 EAST 4TH STREET,10009,28,3,31,4616,18690,1900,2,C7,-,2016-12-14 00:00:00
2,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,39,C7,197 EAST 3RD STREET,10009,16,1,17,2212,7803,1900,2,C7,-,2016-12-09 00:00:00
3,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,402,21,C4,154 EAST 7TH STREET,10009,10,0,10,2272,6794,1913,2,C4,3936272,2016-09-23 00:00:00
4,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,404,55,C2,301 EAST 10TH STREET,10009,6,0,6,2369,4615,1900,2,C2,8000000,2016-11-17 00:00:00


#### Date Format

In [288]:
date = []
for i in df['SALE DATE']:
    date.append(re.findall('\d+', i))
temp1 = pd.DataFrame(date)
temp2 = ({
    'Year' : temp1[0],
    'Month' : temp1[1],
    'Day' : temp1[2]
})
df['SALE DATE'] = pd.to_datetime(temp2)

#### Initially Replacing Missing Values

In [289]:
df['LAND SQUARE FEET'] = df['LAND SQUARE FEET'].replace('-','0',regex=True)
df['GROSS SQUARE FEET'] = df['GROSS SQUARE FEET'].replace('-','0',regex=True)
df['SALE PRICE'] = df['SALE PRICE'].replace('-','-1',regex=True)

#### Data Types

In [290]:
df['TAX CLASS AT TIME OF SALE'] = df['TAX CLASS AT TIME OF SALE'].astype(str)
df['LAND SQUARE FEET'] = df['LAND SQUARE FEET'].astype('int64')
df['GROSS SQUARE FEET'] = df['GROSS SQUARE FEET'].astype('int64')
df['SALE PRICE'] = df['SALE PRICE'].astype('int64')
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [291]:
df.dtypes

BOROUGH                                   object
NEIGHBORHOOD                              object
BUILDING CLASS CATEGORY                   object
TAX CLASS AT PRESENT                      object
BLOCK                                      int64
LOT                                        int64
BUILDING CLASS AT PRESENT                 object
ADDRESS                                   object
ZIP CODE                                   int64
RESIDENTIAL UNITS                          int64
COMMERCIAL UNITS                           int64
TOTAL UNITS                                int64
LAND SQUARE FEET                           int64
GROSS SQUARE FEET                          int64
YEAR BUILT                                 int64
TAX CLASS AT TIME OF SALE                 object
BUILDING CLASS AT TIME OF SALE            object
SALE PRICE                                 int64
SALE DATE                         datetime64[ns]
dtype: object

#### Dropping Rows
- Sale Prices that are missing
- Sale Prices of the properties that are transfer of deeds

In [292]:
df = df.drop(df['SALE PRICE'][df['SALE PRICE'] == -1].index, axis=0)
df = df.drop(df['SALE PRICE'][df['SALE PRICE'] < 1000].index, axis=0)
df.shape

(58681, 19)

In [293]:
print("Sales Dates: ", df['SALE DATE'].min(), " to ", df['SALE DATE'].max())

Sales Dates:  2016-09-01 00:00:00  to  2017-08-31 00:00:00


#### Replacing Invalid and Missing Values

In [294]:
print(df['YEAR BUILT'][(df['YEAR BUILT'] < 1624) & (df['YEAR BUILT'] != 0)])
df['YEAR BUILT'][df['YEAR BUILT'] == 0].count()

957    1111
Name: YEAR BUILT, dtype: int64


4192

In [295]:
df['YEAR BUILT'] = df['YEAR BUILT'].replace(1111, df['YEAR BUILT'].median)

In [296]:
# Missing Values
# Replacing it by median because mean might give a fractional value and we don't need it much
df['ZIP CODE'] = df['ZIP CODE'].replace(0, df['ZIP CODE'].median)
df['LAND SQUARE FEET'] = df['LAND SQUARE FEET'].replace(0, df['LAND SQUARE FEET'].median)
df['GROSS SQUARE FEET'] = df['GROSS SQUARE FEET'].replace(0, df['GROSS SQUARE FEET'].median)
df['YEAR BUILT'] = df['YEAR BUILT'].replace(0, df['YEAR BUILT'].median)
df['BOROUGH'] = df['BOROUGH'].replace('1', 'Manhattan')
df['BOROUGH'] = df['BOROUGH'].replace('2', 'Bronx')
df['BOROUGH'] = df['BOROUGH'].replace('3', 'Brooklyn')
df['BOROUGH'] = df['BOROUGH'].replace('4', 'Queens')
df['BOROUGH'] = df['BOROUGH'].replace('5', 'Staten Island')

In [297]:
df.reset_index(drop=True, inplace=True)
df.to_pickle('nyc_rolling_sales_cleaned.pkl')